In [ ]:
from torch import device

from training_mlp import x_train_processed_wind, y_train_processed_wind, x_test_processed_wind, y_test_processed_wind

from training_mlp import x_train_processed_temp, y_train_processed_temp, x_test_processed_temp, y_test_processed_temp

from training_mlp import x_train_processed_vsby, y_train_processed_vsby, x_test_processed_vsby, y_test_processed_vsby

import matplotlib.pyplot as plt
import pandas as pd
import torch

In [ ]:
## define module for LSTM nn
class LSTMWeather(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super(LSTMWeather, self).__init__()
        self.lstm = torch.nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.linear = torch.nn.Linear(hidden_size, output_size)

    def forward(self,x):
        out, _ = self.lstm(x)
        out = self.linear(out[:, -1, :])
        return out


In [ ]:
def train_lstm(model, x_train, y_train, loss_function, optimizer, epochs):
    loss_history = []

    for epoch in range(epochs):
        total_loss = 0
        model.train()
        for x, y in x_train:
            x, y = x.to(device), y.to(device)
            predictions = model.forward(x)
            loss = loss_function(predictions, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        average_loss = total_loss / len(x_train)
        loss_history.append(average_loss)
